# Machine Learning Project
# Kansas City Crime Data Deep Dive

### Import Dependancies

In [1]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1)
import pandas as pd
from pandas import datetime
from dateutil import parser
from scipy import stats
import glob
import csv
import os

### Obtain KCMO crime data (Raw data came from data.kcmo.org)

In [2]:
path = r'C:\DataAnalytics\KCMO_Crime_Data\Data'
all_files = glob.glob(os.path.join(path, '*.csv'))

next_df = (pd.read_csv(f) for f in all_files)

combined = pd.concat(next_df, ignore_index=True)
combined.head()

,Address,Age,Area,Beat,City,DVFlag,Description,District,Firearm Used Flag,Firearm Used Flag,...,Report_No,Reported Time,Reported_Date,Reported_Time,Sex,To Time,To_Date,To_Time,Unnamed: 18,Zip Code
0,PROSPECT AV/VICTOR ST,28.0,EPD,333,KANSAS CITY,U,Possession of Drug E,NaN,NaN,N,...,70059279,3:24,10/06/2009,NaN,F,NaN,10/05/2009 12:00:00 AM,23:10,NaN,64128.0
1,1100 LOCUST ST,27.0,CPD,112,KANSAS CITY,U,Justifiable Homicide,NaN,NaN,Y,...,80005443,11:45,02/05/2009,NaN,M,NaN,NaN,NaN,NaN,64106.0
2,100 WHEELING AV,22.0,EPD,315,KANSAS CITY,U,Possession/Sale/Dist,NaN,NaN,N,...,80019629,22:50,06/18/2009,NaN,M,NaN,NaN,NaN,NaN,64129.0
3,E 44 ST/INDIANA AV,NaN,EPD,334,KANSAS CITY,U,Stolen Property OFFE,NaN,NaN,N,...,70060962,18:44,01/28/2009,NaN,U,NaN,NaN,NaN,NaN,64130.0
4,1100 LOCUST ST,28.0,CPD,112,KANSAS CITY,U,Justifiable Homicide,NaN,NaN,Y,...,80005443,11:45,02/05/2009,NaN,M,NaN,NaN,NaN,NaN,64106.0


In [3]:
combined.shape

(1160590, 34)

In [4]:
combined.columns

Index(['Address', 'Age', 'Area', 'Beat', 'City', 'DVFlag', 'Description',
       'District', 'Firearm Used Flag', 'Firearm Used Flag  ', 'From Time',
       'From_Date', 'From_Time', 'IBRS', 'Invl_No', 'Involvement', 'Latitude',
       'Location', 'Location 1', 'Longitude', 'Neighborhood', 'Offense',
       'Race', 'Rep_Dist', 'Report_No', 'Reported Time', 'Reported_Date',
       'Reported_Time', 'Sex', 'To Time', 'To_Date', 'To_Time', 'Unnamed: 18',
       'Zip Code'],
      dtype='object')

In [5]:
only_kc_crime = combined[combined.City.str.contains("KANSAS CITY") == True]
only_kc_crime.rename(columns={'Zip Code':'zip_code'}, inplace=True)

In [6]:
only_kc_crime.shape

(1030072, 34)

In [7]:
only_kc_crime = only_kc_crime[only_kc_crime.Sex.str.contains("NaN") == False]
only_kc_crime = only_kc_crime[only_kc_crime.Sex.str.contains("U") == False]
only_kc_crime = only_kc_crime[only_kc_crime.Race.str.contains("NaN") == False]
only_kc_crime = only_kc_crime[only_kc_crime['Age'] < 91]
only_kc_crime.rename(columns={'Zip Code':'zip_code'}, inplace=True)
only_kc_crime = only_kc_crime[only_kc_crime.zip_code != 99999]

In [8]:
only_kc_crime.shape

(568006, 34)

In [9]:
only_kc_crime = only_kc_crime.drop(['To_Date', 'From_Date', 'To_Time', 'From_Time','From Time', 'To Time'], axis=1)

In [10]:
only_kc_crime.shape

(568006, 28)

In [13]:
victims = only_kc_crime[only_kc_crime.Involvement.str.contains("VIC") == True]
suspects = only_kc_crime[only_kc_crime.Involvement.str.contains("SUS") == True]

In [14]:
victims.shape

(355200, 28)

In [15]:
victims.to_csv('kc_victims.csv', index=False)

In [16]:
suspects.shape

(112480, 28)

In [17]:
suspects.to_csv('kc_suspects.csv', index=False)

# STOP HERE

### Combine CSV Files

### Change time to datetime and extract hour

In [ ]:
kc_crime['Reported_Hour'] = pd.to_datetime(kc_crime['Reported_Time'])

In [ ]:
kc_crime["Reported_Hour"] = kc_crime["Reported_Hour"].dt.floor('h')

In [ ]:
kc_crime['Test'] = kc_crime['Reported_Hour'].dt.hour

In [ ]:
kc_crime.head()

### Rename & drop columns

In [ ]:
# kc_crime_dropped_columns = kc_crime.drop(['Reported_Hour', 'Reported_Time', 'To_Date', 'From_Date', 'To_Time', 'From_Time',
#                           'Offense', 'IBRS', 'Rep_Dist', 'Area', 'Beat', 'Address'], axis=1)

### Filter for male and female victims age 90 and under in KCMO

### Copy kc_crime_real_ages and separate "Location" into 3 columns

In [ ]:
kc_crime_real_ages_copy = kc_crime_real_ages.copy()
kc_crime_real_ages_copy.head(1)

# Split out geo data

In [ ]:
location_only = kc_crime_real_ages_copy['Location'].str[0:-1].str.split('\n', expand=True)
location_only.columns = ("address", "city_zip", "geo")
location_only.head(1)

### Parsing out latitude and longitude

In [ ]:
location_only['geo'] = location_only['geo'].str[1:]
geo_split = location_only['geo'].str[0:].str.split(', ', expand=True)
geo_split.columns = ("Latitude", "Longitude")
geo_split.head(1)

In [ ]:
kc_crime_real_ages_copy.count()

In [ ]:
geo_split.count()

In [ ]:
kcmo_crime_with_nans = pd.concat([kc_crime_real_ages_copy, geo_split], axis=1)
kcmo_crime_with_nans.head(1)

In [ ]:
kcmo_crime_with_nans.count()

In [ ]:
kcmo_crime_no_lat_nans = kcmo_crime_with_nans[kcmo_crime_with_nans.Latitude.str.contains("NaN") == False]
kcmo_crime_no_nans = kcmo_crime_no_lat_nans[kcmo_crime_no_lat_nans.Longitude.str.contains("NaN") == False]
kc_crime_close = kcmo_crime_no_nans.drop(['City', 'DVFlag', 'Invl_No', 'Involvement', 'Firearm Used Flag', 'Location'], axis=1)
kc_crime_close.head(1)

In [ ]:
kc_crime_close.columns = ("Report", "Date", "Crime", "Zip", "Race", "Sex", "Age", "Hour", "Latitude", "Longitude")
kc_crime_close = kc_crime_close.reset_index(drop=True)
kc_crime_close.head()

In [ ]:
kc_crime_close.count()

In [ ]:
# plt.scatter(kc_crime_close.Age, kc_crime_close.Zip)
# plt.show()

In [ ]:
# plt.scatter(kc_crime_close.Longitude, kc_crime_close.Latitude, c=kc_crime_close.Age, 
#            cmap='viridis_r', alpha = 0.8, s=2)
# plt.colorbar()
# plt.show()

In [ ]:
x_axis = np.arange(len(kc_crime_close['Crime']))
tick_locations = [value+0.4 for value in x_axis]

In [ ]:
plt.figure(figsize=(20,3))
plt.bar(kc_crime_close['Crime'], kc_crime_close["Age"], color='r', alpha=0.5, align="edge")
plt.xticks(rotation="vertical")

In [ ]:
# Set x and y limits
plt.xlim(-0.25, len(x_axis))
plt.ylim(0, max(kc_crime_close["Age"])+5)

In [ ]:
# Set a Title and labels
plt.title("Average Age Crime of Victims")
plt.xlabel("Crime")
plt.ylabel("Age")

In [ ]:
# plt.tight_layout()
# plt.savefig("avg_state_rain.png")
plt.show()

### Confirming Clean Data in Excel

In [ ]:
# kc_crime_close.to_csv('kc_crime_close.csv', index=False)

# Confirm above data is good before proceeding

### Apply get_dummies function

In [ ]:
crime_columns = pd.get_dummies(kc_crime_close['Crime'],prefix = "Crime")
kc_crime_close = pd.concat([kc_crime_close, crime_columns], axis=1)
kc_crime_close.drop('Crime', axis=1, inplace=True)
zip_columns = pd.get_dummies(kc_crime_close['Zip'],prefix = "Zip")
kc_crime_close = pd.concat([kc_crime_close, zip_columns], axis=1)
kc_crime_close.drop('Zip', axis=1, inplace=True)
race_columns = pd.get_dummies(kc_crime_close['Race'],prefix = "Race")
kc_crime_close = pd.concat([kc_crime_close, race_columns], axis=1)
kc_crime_close.drop('Race', axis=1, inplace=True)
sex_columns = pd.get_dummies(kc_crime_close['Sex'],prefix = "Sex")
kc_crime_close = pd.concat([kc_crime_close, sex_columns], axis=1)
kc_crime_close.drop('Sex', axis=1, inplace=True)
age_columns = pd.get_dummies(kc_crime_close['Age'],prefix = "Age")
kc_crime_close = pd.concat([kc_crime_close, age_columns], axis=1)
kc_crime_close.drop('Age', axis=1, inplace=True)
date_columns = pd.get_dummies(kc_crime_close['Date'],prefix = "Date")
kc_crime_close = pd.concat([kc_crime_close, date_columns], axis=1)
kc_crime_close.drop('Date', axis=1, inplace=True)
hour_columns = pd.get_dummies(kc_crime_close['Hour'],prefix = "Hour")
kc_crime_close = pd.concat([kc_crime_close, hour_columns], axis=1)
kc_crime_close.drop('Hour', axis=1, inplace=True)

In [ ]:
kc_crime_close.head()